In [65]:
import os
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)
import seaborn as sns
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

In [68]:
mission = pd.read_csv("energy/energy/test.csv", engine="python", encoding="euc-kr")
mission["date_time"] = mission["date_time"].astype("datetime64")
mission["hour"] = mission["date_time"].dt.hour
mission["day"]= mission["date_time"].dt.weekday
mission["Month"] = mission["date_time"].dt.month
mission.drop(columns="date_time",axis=1,inplace=True)
mission[mission.isnull() == True] = 0

,num,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유,hour,day,Month
0,1,27.8,1.5,74.0,0.0,0.0,0.0,0.0,0,1,8
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,8
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,8
3,1,27.3,1.1,78.0,0.0,0.0,0.0,0.0,3,1,8
4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,8
...,...,...,...,...,...,...,...,...,...,...,...
10075,60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19,0,8
10076,60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,0,8
10077,60,27.9,4.1,68.0,0.0,0.0,1.0,1.0,21,0,8
10078,60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22,0,8


In [69]:
# 3(시간) 등차 수열
list1 = ['기온(°C)', '풍속(m/s)', '습도(%)','일조(hr, 3시간)']
for n in np.arange(0, 10079, 168) :
    for i in list1 :
        for x in np.arange(n,n+165,3) :
            mission[i][x+1] =  mission[i][x] +(mission[i][x+3]-mission[i][x])/3
            mission[i][x+2] =  mission[i][x] +(mission[i][x+3]-mission[i][x])*2/3
        mission[i][n+166] = mission[i][n+165]
        mission[i][n+167] = mission[i][n+165]
    print(f"---------------------{(n/168)+1}번째 건물 DATA처리 완료")
# 6(시간) 등차 수열
for n in np.arange(0, 10079, 168) :
    for i in np.arange(n,n+162,6) :
        for x in np.arange(0,5) :
            mission['강수량(mm, 6시간)'][i+x+1] =  mission['강수량(mm, 6시간)'][i+x] +(mission['강수량(mm, 6시간)'][i+6]-mission['강수량(mm, 6시간)'][i])/6
        for x in range(1,6) :
            mission['강수량(mm, 6시간)'][n+162 + x] = mission['강수량(mm, 6시간)'][n+162]
    print(f"---------------------{(n/168)+1}번째 건물 DATA처리 완료")    
mission = mission.round(2).drop(columns=["비전기냉방설비운영","태양광보유"], axis=1)

---------------------1.0번째 건물 DATA처리 완료
---------------------2.0번째 건물 DATA처리 완료
---------------------3.0번째 건물 DATA처리 완료
---------------------4.0번째 건물 DATA처리 완료
---------------------5.0번째 건물 DATA처리 완료
---------------------6.0번째 건물 DATA처리 완료
---------------------7.0번째 건물 DATA처리 완료
---------------------8.0번째 건물 DATA처리 완료
---------------------9.0번째 건물 DATA처리 완료
---------------------10.0번째 건물 DATA처리 완료
---------------------11.0번째 건물 DATA처리 완료
---------------------12.0번째 건물 DATA처리 완료
---------------------13.0번째 건물 DATA처리 완료
---------------------14.0번째 건물 DATA처리 완료
---------------------15.0번째 건물 DATA처리 완료
---------------------16.0번째 건물 DATA처리 완료
---------------------17.0번째 건물 DATA처리 완료
---------------------18.0번째 건물 DATA처리 완료
---------------------19.0번째 건물 DATA처리 완료
---------------------20.0번째 건물 DATA처리 완료
---------------------21.0번째 건물 DATA처리 완료
---------------------22.0번째 건물 DATA처리 완료
---------------------23.0번째 건물 DATA처리 완료
---------------------24.0번째 건물 DATA처리 완료
---------------------25.0

In [71]:
mission

,num,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",hour,day,Month
0,1,27.80,1.50,74.00,0.0,0.00,0,1,8
1,1,27.63,1.37,75.33,0.0,0.00,1,1,8
2,1,27.47,1.23,76.67,0.0,0.00,2,1,8
3,1,27.30,1.10,78.00,0.0,0.00,3,1,8
4,1,26.90,1.17,79.67,0.0,0.00,4,1,8
...,...,...,...,...,...,...,...,...,...
10075,60,28.63,3.57,66.00,0.0,0.53,19,0,8
10076,60,28.27,3.83,67.00,0.0,0.27,20,0,8
10077,60,27.90,4.10,68.00,0.0,0.00,21,0,8
10078,60,27.90,4.10,68.00,0.0,0.00,22,0,8


In [72]:
mission.to_csv("mission.csv", encoding="utf-8")